# Chapter 9

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [2]:
from statsmodels.stats.diagnostic import linear_reset

# Exercise 1
ceosal1 = pd.read_stata("stata/CEOSAL1.DTA")
ceosal1["rosneg"] = (ceosal1.ros < 0).astype('int32')

y = ceosal1.lsalary
X = sm.add_constant(ceosal1[["lsales", "roe", "rosneg"]])
model = sm.OLS(y, X).fit()
print(linear_reset(model))

<Wald test (chi2): statistic=[[2.66707016]], p-value=0.26354396347186887, df_denom=2>


In [3]:
linear_reset(model, cov_type = "HC3")

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[2.99859337]], p-value=0.22328714637257752, df_denom=2>

C1.i The p-value for the RESET test is about 0.26 which could not be considered evidence of functional form misspecification.

C1.ii When considering a heteroskedacticity robust RESET the p-value falls to about 0.22, but this is far from any value we would consider evidence of misspecification.

In [4]:
# Exercise 2
wage2 = pd.read_stata("stata/WAGE2.DTA")
wage2["educ_iq"] = wage2.educ * wage2.IQ
wage2["educ_kww"] = wage2.educ * wage2.KWW

y = wage2.lwage
X = sm.add_constant(wage2[["educ", "exper", "tenure", "married", "south", "urban", "black", "KWW", "educ_kww"]])
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     35.97
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           8.91e-55
Time:                        15:32:18   Log-Likelihood:                -377.34
No. Observations:                 935   AIC:                             774.7
Df Residuals:                     925   BIC:                             823.1
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3557      0.114     47.113      0.0

In [5]:
wage2["combined"] = wage2.IQ * wage2.KWW
wage2["educ_combined"] = wage2.combined * wage2.educ
X = sm.add_constant(wage2[["educ", "exper", "tenure", "married", "south", "urban", "black", "IQ", "KWW", "educ_combined"]])
model = sm.OLS(y, X).fit()
print(model.summary())
print(model.f_test("(IQ = 0), (KWW = 0)"))

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     33.53
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           7.48e-56
Time:                        15:32:18   Log-Likelihood:                -372.88
No. Observations:                 935   AIC:                             767.8
Df Residuals:                     924   BIC:                             821.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             5.1756      0.128     40.486

C2.i The estimated return to education when using KWW as a proxy for ability is about 5.8%

C2.ii The return to education falls to about 4.9%

C.iii Both IQ and KWW are significant at the 5% level (in fact, IQ is significant at the 1% level) individually. The F-test provides strong evidence that they are jointly significant.

In [6]:
# Exercise 3
jtrain = pd.read_stata("stata/JTRAIN.DTA")
jtrain_1988 = jtrain.loc[jtrain["year"] == 1988, ["lscrap", "grant"]].dropna()
y = jtrain_1988.lscrap
X = sm.add_constant(jtrain_1988.grant)

model = sm.OLS(y, X).fit()
print(model.summary())
print(model.get_robustcov_results(cov_type = "HC3").summary())

                            OLS Regression Results                            
Dep. Variable:                 lscrap   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                   0.01948
Date:                Sat, 06 Jun 2020   Prob (F-statistic):              0.890
Time:                        15:32:18   Log-Likelihood:                -94.660
No. Observations:                  54   AIC:                             193.3
Df Residuals:                      52   BIC:                             197.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4085      0.241      1.698      0.0

In [7]:
jtrain["lscrap_lag"] = jtrain["lscrap"].shift(1)
jtrain_1988 = jtrain.loc[jtrain["year"] == 1988, ["lscrap", "lscrap_lag", "grant"]].dropna()
y = jtrain_1988.lscrap
X = sm.add_constant(jtrain_1988[["grant", "lscrap_lag"]])

model = sm.OLS(y, X).fit()
model_hc3 = model.get_robustcov_results(cov_type = "HC3")
print(model.summary())
print("One sided p-value for grant is: ", (model.pvalues[1] / 2))
print(model_hc3.summary())
print("One sided p-value for grant is: ", (model_hc3.pvalues[1] / 2))

                            OLS Regression Results                            
Dep. Variable:                 lscrap   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     174.9
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           1.47e-23
Time:                        15:32:18   Log-Likelihood:                -39.000
No. Observations:                  54   AIC:                             84.00
Df Residuals:                      51   BIC:                             89.97
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0212      0.089      0.238      0.8

In [8]:
print(model.t_test("lscrap_lag = 1"))
print(model_hc3.t_test("lscrap_lag = 1"))

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.8312      0.044     -3.799      0.000       0.742       0.920
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.8312      0.088     -1.914      0.061       0.654       1.008


C3.i Quite a lot depends on how grants are assigned. It would be fair to say that grants can be assigned to over or under performing firms, depending on who is administering the program. In either case the factors leading to higher or lower productivity are unobserved but would be correlated with grant.

C3.ii The simple regression does not provide evidence that receiving a job training grant significantly lowers a firm's scrap rate. With a positive coefficient, a significant result would mean the grant increased the scrap rate!

C3.iii Adding the lag turns the coefficient for grant negative. It is statistically significant at the 5% level with a p-value of 0.045 against the one sided test $\beta_{grant} < 0$.

C3.iv The t-test is listed above and the p-value is incredibly small (0.000).

C3.v Robust results are reported below the standard results. The results from part (ii) do not substantively change. While the question did not ask to check part (iii), grant is now only significant at the 10% level when tested against the one sided alternative. The robust standard error is quite a bit larger and the test is now significant only at the 10% level.

In [9]:
# Exercise 4
infmrt = pd.read_stata("stata/infmrt.dta")
infmrt_1990 = infmrt[infmrt.year == 1990]
y = infmrt_1990.infmort
X = sm.add_constant(infmrt_1990[["lpcinc", "lphysic", "lpopul", "DC"]])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                infmort   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     25.71
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           3.15e-11
Time:                        15:32:18   Log-Likelihood:                -80.968
No. Observations:                  51   AIC:                             171.9
Df Residuals:                      46   BIC:                             181.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.9548     12.419      1.929      0.0

C4.i The coefficient indicates that once income, number of physicians, and population are accounted for, DC suffers 16 more infant deaths per 1000 live births. This is a large and significant effect.

C4.ii The coefficients are identical to 9.44 (on page 331). Including the dummy accounts for the distortion created by the outlier. This is only true for a single observation.

In [10]:
# Exercise 5
rdchem = pd.read_stata("stata/RDCHEM.DTA")
rdchem["sales"]  = rdchem["sales"] / 1000
rdchem["salessq"] = np.power(rdchem["sales"], 2)
rdchem_limit = rdchem[rdchem.sales < 20]
y = rdchem.rdintens
X = sm.add_constant(rdchem[["sales", "salessq", "profmarg"]])

model = sm.OLS(y, X).fit()
print(model.summary())

y = rdchem_limit.rdintens
X = sm.add_constant(rdchem_limit[["sales", "salessq", "profmarg"]])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               rdintens   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     2.196
Date:                Sat, 06 Jun 2020   Prob (F-statistic):              0.111
Time:                        15:32:18   Log-Likelihood:                -61.610
No. Observations:                  32   AIC:                             131.2
Df Residuals:                      28   BIC:                             137.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0590      0.626      3.288      0.0

In [11]:
y = rdchem.rdintens
X = sm.add_constant(rdchem[["sales", "salessq", "profmarg"]])
model = sm.QuantReg(y, X).fit(q = .5)
print(model.summary())

y = rdchem_limit.rdintens
X = sm.add_constant(rdchem_limit[["sales", "salessq", "profmarg"]])
model = sm.QuantReg(y, X).fit(q = .5)
print(model.summary())

                         QuantReg Regression Results                          
Dep. Variable:               rdintens   Pseudo R-squared:              0.09743
Model:                       QuantReg   Bandwidth:                       2.034
Method:                 Least Squares   Sparsity:                        4.266
Date:                Sat, 06 Jun 2020   No. Observations:                   32
Time:                        15:32:18   Df Residuals:                       28
                                        Df Model:                            3
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4043      0.753      1.865      0.073      -0.138       2.947
sales          0.2635      0.167      1.578      0.126      -0.079       0.605
salessq       -0.0060      0.004     -1.343      0.190      -0.015       0.003
profmarg       0.1140      0.053      2.144      0.0

C5.i The largest changes are in sales, with sales changing by 0.05 and the quadratic term falling a further 0.003. More noteably, sales was significant at the 5% level and the quadratic was significant at the 10% level while removing the outlier removes all significance.

C5.ii Removing the outlier flips the sign of the sales coefficients! standard errors grow, but none of the results were significant in the first place.

C5.iii The text introduced LAD as being resilient to outliers. I am not at all confident in a method that flips signs depending on the inclusion of a variable, but one expectation was that we would have a large sample, and we are falling from 32 to 31 observations. Based on these results, I would say OLS is more resilient to outliers.

In [12]:
# Exercise 6
meap93 = pd.read_stata("stata/MEAP93.DTA")
meap93_limit = meap93[meap93.bensal > 0.01]
print(meap93.shape[0] - meap93_limit.shape[0])

4


In [13]:
y = meap93_limit.lsalary
X = sm.add_constant(meap93_limit[["bensal", "lenroll", "lstaff", "droprate", "gradrate"]])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                lsalary   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     43.55
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           8.75e-36
Time:                        15:32:18   Log-Likelihood:                 272.94
No. Observations:                 404   AIC:                            -533.9
Df Residuals:                     398   BIC:                            -509.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.7053      0.260     41.119      0.0

C6.i Four of the observations have a benefits/salary ratio of less than 0.01

C6.ii The magnitude of bensal (b/s in the text) is quite a bit smaller than in Example 4.10 (page 156), shrinking from -0.589 to -0.421. The standard error has also grown, though the result is still significant at the 5% level. The remaining coefficients are consistent.

In [14]:
# Exercise 7
loanapp = pd.read_stata("stata/loanapp.dta")
print(loanapp[loanapp.obrat > 40].shape[0])

205


In [15]:
loanapp_reg = loanapp.loc[loanapp.obrat <= 40, ["approve", "white", "hrat", "obrat", "loanprc", "unem", "male", "married", "dep", "sch", "cosign", "chist", "pubrec", "mortlat1", "mortlat2", "vr"]].dropna()

y = loanapp_reg.approve
X = sm.add_constant(loanapp_reg[["white", "hrat", "obrat", "loanprc", "unem", "male", "married", "dep", "sch", "cosign", "chist", "pubrec", "mortlat1", "mortlat2", "vr"]])
model = sm.OLS(y, X).fit()
model_summary = model.summary()
print(model_summary)

                            OLS Regression Results                            
Dep. Variable:                approve   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     17.35
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           2.63e-43
Time:                        15:32:18   Log-Likelihood:                -249.57
No. Observations:                1768   AIC:                             531.1
Df Residuals:                    1752   BIC:                             618.8
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7872      0.055     14.377      0.0

C7.i 205 observations have other debt obligations in excess of 40% of their income

C7.ii The coefficient for white is consistent, moving from 0.1288 to 0.1286. The t statistic shrinks from 6.529 to 6.508. These are not large changes.

C7.iii It does not appear that the estimate for $\beta_{white}$ is overly sensitive to the sample used (at least with regards to the debt obligations)

In [16]:
# Exercise 8
twoyear = pd.read_stata("stata/twoyear.dta")
print("The sample mean of stotal is:", twoyear.stotal.mean())
print("The standard deviation is:", twoyear.stotal.std())

The sample mean of stotal is: 0.04748285189270973
The standard deviation is: 0.8535629510879517


In [17]:
y = twoyear.stotal
X = sm.add_constant(twoyear.jc)

model = sm.OLS(y, X).fit()
print(model.summary())

X = sm.add_constant(twoyear.univ)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 stotal   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.032
Date:                Sat, 06 Jun 2020   Prob (F-statistic):              0.310
Time:                        15:32:18   Log-Likelihood:                -8524.3
No. Observations:                6763   AIC:                         1.705e+04
Df Residuals:                    6761   BIC:                         1.707e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0429      0.011      3.781      0.0

In [18]:
y = twoyear.lwage
X = sm.add_constant(twoyear[["jc", "univ", "exper", "stotal"]])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     500.2
Date:                Sat, 06 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:32:18   Log-Likelihood:                -3862.5
No. Observations:                6763   AIC:                             7735.
Df Residuals:                    6758   BIC:                             7769.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4953      0.021     70.473      0.0

In [19]:
twoyear["stotal_sq"] = np.power(twoyear.stotal, 2)

X = sm.add_constant(twoyear[["jc", "univ", "exper", "stotal", "stotal_sq"]])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     400.2
Date:                Sat, 06 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:32:19   Log-Likelihood:                -3862.4
No. Observations:                6763   AIC:                             7737.
Df Residuals:                    6757   BIC:                             7778.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4940      0.021     69.635      0.0

In [20]:
twoyear["stotal_jc"] = twoyear.stotal * twoyear.jc
twoyear["stotal_univ"] = twoyear.stotal * twoyear.univ

X = sm.add_constant(twoyear[["jc", "univ", "exper", "stotal", "stotal_jc", "stotal_univ"]])

model = sm.OLS(y, X).fit()
print(model.summary())
print(model.f_test("(stotal_jc = 0, stotal_univ = 0)"))

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     334.2
Date:                Sat, 06 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:32:19   Log-Likelihood:                -3860.5
No. Observations:                6763   AIC:                             7735.
Df Residuals:                    6756   BIC:                             7783.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.4959      0.021     70.419      

C8.i The mean is 0.047 and the standard deviation is 0.854.

C8.ii Only univ is statistically related to stotal. We know this because of the significant relationship between univ and stotal (while jc small and not significant).

C8.iii The effect of univ is smaller and more precise.

C8.iv Adding $stotal^2$ produces a small coefficient that is not significantly different from 0.

C8.v The interaction terms are not jointly significant even at the 10% level.

C8.vi The model from part iii should be enough. We did not find any evidence that any of the alternatives explored in iv or v added anything to the analysis.

In [21]:
# Exercise 9

k401subs = pd.read_stata("stata/401ksubs.dta")
y = k401subs.nettfa
X = sm.add_constant(k401subs[["inc", "incsq", "age", "agesq", "male", "e401k"]])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 nettfa   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     391.6
Date:                Sat, 06 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:32:19   Log-Likelihood:                -50681.
No. Observations:                9275   AIC:                         1.014e+05
Df Residuals:                    9268   BIC:                         1.014e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.1978      9.992      2.121      0.0

In [22]:
import statsmodels.stats.diagnostic as smd
print(smd.het_breuschpagan(model.resid, X))

(346.6439627872171, 8.107902005744488e-72, 59.97177669516518, 3.338601184163747e-73)


In [23]:
model = sm.QuantReg(y, X).fit(q = .5)
print(model.summary())

                         QuantReg Regression Results                          
Dep. Variable:                 nettfa   Pseudo R-squared:               0.1088
Model:                       QuantReg   Bandwidth:                       2.514
Method:                 Least Squares   Sparsity:                        16.54
Date:                Sat, 06 Jun 2020   No. Observations:                 9275
Time:                        15:32:19   Df Residuals:                     9268
                                        Df Model:                            6
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.4912      1.446      8.637      0.000       9.656      15.326
inc           -0.2616      0.011    -24.221      0.000      -0.283      -0.240
incsq          0.0071    8.5e-05     83.381      0.000       0.007       0.007
age           -0.7227      0.070    -10.327      0.0

C9.i e401k indicates that on average people who are eligible for a 401k have \$9,713 higher financial assets.

C9.ii There is strong evidence of heteroskedasticity. The p-value is near zero (F statistic of 59.97)

C9.iii LAD produces a noticeably smaller coefficient for e401k. This would mean the median individiual eligible for a 401k has about \$3,737 more in financial assets.

C9.iv The difference between the two mark the difference between mean and median. Given that we are considering financial assets it makes sense that the mean would be higher than the median (as with income). We would expect futher investigation to show a skewed distribution.

In [24]:
# Exercise 10

jtrain2 = pd.read_stata("stata/JTRAIN2.DTA")
jtrain3 = pd.read_stata("stata/jtrain3.dta")

print(jtrain2.train.sum() / jtrain2.shape[0], "of jtrain2 received job training")
print(jtrain3.train.sum() / jtrain3.shape[0], "of jtrain3 received job training")

0.4157303370786517 of jtrain2 received job training
0.0691588785046729 of jtrain3 received job training


In [25]:
y = jtrain2.re78
X = sm.add_constant(jtrain2[["train"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     8.039
Date:                Sat, 06 Jun 2020   Prob (F-statistic):            0.00479
Time:                        15:32:19   Log-Likelihood:                -1468.8
No. Observations:                 445   AIC:                             2942.
Df Residuals:                     443   BIC:                             2950.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.5548      0.408     11.162      0.0

In [26]:
X = sm.add_constant(jtrain2[["train", "re74", "re75", "educ", "age", "black", "hisp"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     3.617
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           0.000840
Time:                        15:32:19   Log-Likelihood:                -1460.3
No. Observations:                 445   AIC:                             2937.
Df Residuals:                     437   BIC:                             2969.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6741      2.423      0.278      0.7

In [27]:
y = jtrain3.re78
X = sm.add_constant(jtrain3[["train"]])
model = sm.OLS(y, X).fit()

print(model.summary())

X = sm.add_constant(jtrain3[["train", "re74", "re75", "educ", "age", "black", "hisp"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     173.4
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           2.03e-38
Time:                        15:32:19   Log-Likelihood:                -11066.
No. Observations:                2675   AIC:                         2.214e+04
Df Residuals:                    2673   BIC:                         2.215e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.5539      0.304     70.985      0.0

In [28]:
jtrain2["avgre"] = (jtrain2.re74 + jtrain2.re75) / 2
jtrain3["avgre"] = (jtrain3.re74 + jtrain3.re75) / 2

print("jtrain2 avgre")
print("Avg.", jtrain2.avgre.mean())
print("S.d.", jtrain2.avgre.std())
print("Min.", jtrain2.avgre.min())
print("Max.", jtrain2.avgre.max())

print("jtrain3 avgre")
print("Avg.", jtrain3.avgre.mean())
print("S.d.", jtrain3.avgre.std())
print("Min.", jtrain3.avgre.min())
print("Max.", jtrain3.avgre.max())

jtrain2 avgre
Avg. 1.7397023439407349
S.d. 3.9000966548919678
Min. 0.0
Max. 24.37645149230957
jtrain3 avgre
Avg. 18.040435791015625
S.d. 13.293451309204102
Min. 0.0
Max. 146.9010009765625


In [29]:
jtrain2_limited = jtrain2[jtrain2.avgre <= 10]

y = jtrain2_limited.re78
X = sm.add_constant(jtrain2_limited[["train", "re74", "re75", "educ", "age", "black", "hisp"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     2.904
Date:                Sat, 06 Jun 2020   Prob (F-statistic):            0.00564
Time:                        15:32:20   Log-Likelihood:                -1393.0
No. Observations:                 427   AIC:                             2802.
Df Residuals:                     419   BIC:                             2834.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7369      2.446      0.710      0.4

In [30]:
jtrain3_limited = jtrain3[jtrain3.avgre <= 10]

y = jtrain3_limited.re78
X = sm.add_constant(jtrain3_limited[["train", "re74", "re75", "educ", "age", "black", "hisp"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     33.11
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           2.89e-40
Time:                        15:32:20   Log-Likelihood:                -2668.6
No. Observations:                 765   AIC:                             5353.
Df Residuals:                     757   BIC:                             5390.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.4480      2.141      1.610      0.1

In [31]:
jtrain2_limited = jtrain2[(jtrain2.unem74 == 1) & (jtrain2.unem75 == 1)]

y = jtrain2_limited.re78
X = sm.add_constant(jtrain2_limited[["train"]])
model = sm.OLS(y, X).fit()

print(model.summary())

jtrain3_limited = jtrain3[(jtrain3.unem74 == 1) & (jtrain3.unem75 == 1)]

y = jtrain3_limited.re78
X = sm.add_constant(jtrain3_limited[["train"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     7.144
Date:                Sat, 06 Jun 2020   Prob (F-statistic):            0.00797
Time:                        15:32:20   Log-Likelihood:                -879.83
No. Observations:                 280   AIC:                             1764.
Df Residuals:                     278   BIC:                             1771.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1124      0.430      9.563      0.0

C10.i 41.5% of jtrain2 participated in training while only 6.9% of jtrain3 participated. This indicates the difference between the experiment (which would ideally have 50%) and self selection.

C10.ii The estimated effect of job trainign on earnings is \$1,790 (in real terms).

C10.iii The difference after the controls is about \$100 less, which is not a very large effect. This makes sense since this is experimental data and so it is reasonable to assume that the controls that were previously in the error term were uncorrelated with the training variable.

C10.iv The simple regression implies the training program reduces real earnings by \\$15,204, while the regression with controls, while controls produce an estimate of about \$200 (and not statistically significant). We have already remarked that this sample is self-selected and we are seeing the specific consequence of low earners choosing job training. The difference between the two estimates is attributible to factors that were unaccounted for in the simple regression.

C10.v jtrain2 has an average of 1.74, a standard deviation of 3.90, a miin of 0 and a max of 24.38. jtrain3 has an average of 18.04, standard deviation of of 13.29, a min of 0 and a max of 146.90. Both cannot be representative.

C10.vi For jtrain2 the effect is about \\$1,580, not to different from before. From jtrain3 the effect is about \$1,845, quite a bit closer to the estimate from the experiment.

C10.vii jtrain2's estimate is 1.84 compared to jtrain3's 3.80.

C10.viii The effects are closer when we have comparable samples (such as the average real income case). The divergence between the two data sets came from the smaller proportion of people looking for a job, entangling the effect of the training program with the effect of unemployment.

In [32]:
# Exercise 11
murder = pd.read_stata("stata/MURDER.DTA")
murder["mrdrte_lag"] = murder.mrdrte.shift(1)
murder_1993 = murder[murder.year == 93]

In [33]:
y = murder_1993.mrdrte
X = sm.add_constant(murder_1993[["exec", "unem"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     3.126
Date:                Sat, 06 Jun 2020   Prob (F-statistic):             0.0529
Time:                        15:32:20   Log-Likelihood:                -189.70
No. Observations:                  51   AIC:                             385.4
Df Residuals:                      48   BIC:                             391.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.5686      6.315     -1.040      0.3

In [34]:
murder_1993.loc[murder_1993.state  == "TX", "exec"]

131    34
Name: exec, dtype: int8

In [35]:
murder_1993["texas"] = (murder_1993.state == "TX").astype("int")
X = sm.add_constant(murder_1993[["exec", "unem", "texas"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     2.079
Date:                Sat, 06 Jun 2020   Prob (F-statistic):              0.116
Time:                        15:32:20   Log-Likelihood:                -189.65
No. Observations:                  51   AIC:                             387.3
Df Residuals:                      47   BIC:                             395.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.8265      6.426     -1.062      0.2

/home/systemchalk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
X = sm.add_constant(murder_1993[["exec", "unem", "mrdrte_lag"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1609.
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           2.34e-47
Time:                        15:32:20   Log-Likelihood:                -74.461
No. Observations:                  51   AIC:                             156.9
Df Residuals:                      47   BIC:                             164.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8826      0.676      1.306      0.1

In [37]:
X = sm.add_constant(murder_1993[["exec", "unem", "mrdrte_lag", "texas"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1185.
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           9.55e-46
Time:                        15:32:21   Log-Likelihood:                -74.383
No. Observations:                  51   AIC:                             158.8
Df Residuals:                      46   BIC:                             168.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8491      0.688      1.234      0.2

In [38]:
murder_1993_notx = murder_1993[murder.state != "TX"]

y = murder_1993_notx.mrdrte
X = sm.add_constant(murder_1993_notx[["exec", "unem", "mrdrte_lag"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1577.
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           2.29e-46
Time:                        15:32:21   Log-Likelihood:                -73.420
No. Observations:                  50   AIC:                             154.8
Df Residuals:                      46   BIC:                             162.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8491      0.688      1.234      0.2

/home/systemchalk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


C11.i The coefficient is small (and positive) and statistically indistinguishable from zero. This is not evidence of a deterrent effect for capital punishment.

C11.ii There were 34 executions reported for Texas. The t-statistic from adding it to the regression is not large and it does not appear to be an outlier.

C11.iii The coefficient $\hat{\beta}_{exec}$ flips to a negative value and is now significant. This would suggest a deterrent effect.

C11.iv When adding a Texas dummy the coefficient for exec is no longer significant, but texas itself is not significant and our aim is to see if it is an outlier. From this, we do not have evidence it is an outlier. Dropping it from the regression produces the same coefficients as including the dummy (as expected) and we see that we are no longer able to precisely estimate a deterrent effect by dropping Texas.

In [39]:
# Exercise 13
elem94_95 = pd.read_stata("stata/elem94_95.dta")

y = elem94_95.lavgsal
X = sm.add_constant(elem94_95[["bs", "lenrol", "lstaff", "lunch"]])
model = sm.OLS(y, X).fit()

print(model.summary())
print(model.get_robustcov_results(cov_type = "HC3").summary())

                            OLS Regression Results                            
Dep. Variable:                lavgsal   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.487
Method:                 Least Squares   F-statistic:                     439.4
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          4.22e-266
Time:                        15:32:21   Log-Likelihood:                 689.98
No. Observations:                1848   AIC:                            -1370.
Df Residuals:                    1843   BIC:                            -1342.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.8315      0.110    126.055      0.0

In [40]:
elem94_95_limit = elem94_95[elem94_95.bs <= .5]

y = elem94_95_limit.lavgsal
X = sm.add_constant(elem94_95_limit[["bs", "lenrol", "lstaff", "lunch"]])
model = sm.OLS(y, X).fit()

print(model.summary())
print(model.get_robustcov_results(cov_type = "HC3").summary())

                            OLS Regression Results                            
Dep. Variable:                lavgsal   R-squared:                       0.489
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     439.7
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          4.58e-266
Time:                        15:32:21   Log-Likelihood:                 707.02
No. Observations:                1844   AIC:                            -1404.
Df Residuals:                    1839   BIC:                            -1376.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.7101      0.111    123.071      0.0

In [41]:
elem94_95.bs[elem94_95.bs > .5]

67      0.659488
1126    0.574776
1507    0.999785
1669    0.502258
Name: bs, dtype: float32

In [42]:
elem94_95["d68"] = 0
elem94_95["d1127"] = 0
elem94_95["d1508"] = 0
elem94_95["d1670"] = 0

elem94_95.loc[67, "d68"] = 1
elem94_95.loc[1126, "d1127"] = 1
elem94_95.loc[1507, "d1508"] = 1
elem94_95.loc[1669, "d1670"] = 1

In [43]:
y = elem94_95.lavgsal
X = sm.add_constant(elem94_95[["bs", "lenrol", "lstaff", "lunch", "d68", "d1127", "d1508", "d1670"]])
model = sm.OLS(y, X).fit()

print(model.summary())
print(model.get_robustcov_results(cov_type = "HC3").summary())

                            OLS Regression Results                            
Dep. Variable:                lavgsal   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                     229.3
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          7.48e-270
Time:                        15:32:21   Log-Likelihood:                 710.55
No. Observations:                1848   AIC:                            -1403.
Df Residuals:                    1839   BIC:                            -1353.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.7101      0.111    123.071      0.0

In [44]:
elem94_95_limit = elem94_95[elem94_95.d1508 == 0]

y = elem94_95_limit.lavgsal
X = sm.add_constant(elem94_95_limit[["bs", "lenrol", "lstaff", "lunch"]])
model = sm.OLS(y, X).fit()

print(model.summary())

elem94_95_limit = elem94_95[elem94_95.d68 == 0]

y = elem94_95_limit.lavgsal
X = sm.add_constant(elem94_95_limit[["bs", "lenrol", "lstaff", "lunch"]])
model = sm.OLS(y, X).fit()

print(model.summary())

elem94_95_limit = elem94_95[elem94_95.d1127 == 0]

y = elem94_95_limit.lavgsal
X = sm.add_constant(elem94_95_limit[["bs", "lenrol", "lstaff", "lunch"]])
model = sm.OLS(y, X).fit()

print(model.summary())

elem94_95_limit = elem94_95[elem94_95.d1670 == 0]

y = elem94_95_limit.lavgsal
X = sm.add_constant(elem94_95_limit[["bs", "lenrol", "lstaff", "lunch"]])
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                lavgsal   R-squared:                       0.489
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     441.4
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          6.17e-267
Time:                        15:32:21   Log-Likelihood:                 708.02
No. Observations:                1847   AIC:                            -1406.
Df Residuals:                    1842   BIC:                            -1378.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.7096      0.110    124.163      0.0

In [45]:
y = elem94_95.lavgsal
X = sm.add_constant(elem94_95[["bs", "lenrol", "lstaff", "lunch"]])

model = sm.QuantReg(y, X).fit(q = .5)

print(model.summary())

elem94_95_limit = elem94_95[elem94_95.d1508 == 0]

y = elem94_95_limit.lavgsal
X = sm.add_constant(elem94_95_limit[["bs", "lenrol", "lstaff", "lunch"]])
model = sm.QuantReg(y, X).fit(q = .5)

print(model.summary())

                         QuantReg Regression Results                          
Dep. Variable:                lavgsal   Pseudo R-squared:               0.2893
Model:                       QuantReg   Bandwidth:                     0.06254
Method:                 Least Squares   Sparsity:                       0.3878
Date:                Sat, 06 Jun 2020   No. Observations:                 1848
Time:                        15:32:21   Df Residuals:                     1843
                                        Df Model:                            4
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.9873      0.128    109.664      0.000      13.737      14.237
bs            -0.1091      0.128     -0.855      0.393      -0.359       0.141
lenrol        -0.0572      0.010     -5.823      0.000      -0.077      -0.038
lstaff        -0.7217      0.021    -33.814      0.0

C12.i The estimate is significant and economically significant in the first model, but the heteroskedasticity-robust errors mean the significance falls.

C12.ii The coefficient is much smaller and is no longer statistically significant.

C12.iii The OLS estimates are the same from the dropped version when we include dummies. The observation 1508 is statistically significant (indexing in Python starts at 0 so this is in position 1507)

C12.iv The values of the coefficients can be confirmed above (they match). Only dropping observation 1508 (index 1507) produces a substantial difference.

C12.v OLS is still sensitive to a single observation, even when there are lots of observations.

C12.vi In contrast to OLS, LAD is less sensitive to the outlier with the coefficient of -0.109 shrinking to -0.097 and both not being statistically significant

In [46]:
# Exercise 13
ceosal2 = pd.read_stata("stata/CEOSAL2.DTA")

y = ceosal2.lsalary
X = sm.add_constant(ceosal2[["lsales", "lmktval", "ceoten", "ceotensq"]])
model = sm.OLS(y, X).fit()
ols_b = model.params

print(model.summary())
print(model.get_robustcov_results(cov_type = "HC3").summary())

                            OLS Regression Results                            
Dep. Variable:                lsalary   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     22.45
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           6.26e-15
Time:                        15:32:21   Log-Likelihood:                -124.84
No. Observations:                 177   AIC:                             259.7
Df Residuals:                     172   BIC:                             275.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.3686      0.259     16.884      0.0

In [47]:
from statsmodels.stats.outliers_influence import OLSInfluence

ceosal2["str"] = OLSInfluence(model).resid_studentized
(abs(ceosal2["str"]) > 1.96).sum()

9

In [48]:
ceosal2_limit = ceosal2[abs(ceosal2.str) < 1.96]

y = ceosal2_limit.lsalary
X = sm.add_constant(ceosal2_limit[["lsales", "lmktval", "ceoten", "ceotensq"]])
model = sm.OLS(y, X).fit()
lim_b = model.params

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                lsalary   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     41.43
Date:                Sat, 06 Jun 2020   Prob (F-statistic):           6.26e-24
Time:                        15:32:21   Log-Likelihood:                -72.748
No. Observations:                 168   AIC:                             155.5
Df Residuals:                     163   BIC:                             171.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1381      0.204     20.302      0.0

In [49]:
y = ceosal2.lsalary
X = sm.add_constant(ceosal2[["lsales", "lmktval", "ceoten", "ceotensq"]])
model = sm.QuantReg(y, X).fit(q = .5)
lad_b = model.params

print(model.summary())

                         QuantReg Regression Results                          
Dep. Variable:                lsalary   Pseudo R-squared:               0.2668
Model:                       QuantReg   Bandwidth:                      0.3845
Method:                 Least Squares   Sparsity:                        1.155
Date:                Sat, 06 Jun 2020   No. Observations:                  177
Time:                        15:32:21   Df Residuals:                      172
                                        Df Model:                            4
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1343      0.301     13.753      0.000       3.541       4.728
lsales         0.1492      0.045      3.324      0.001       0.061       0.238
lmktval        0.1529      0.057      2.696      0.008       0.041       0.265
ceoten         0.0431      0.016      2.629      0.0

In [51]:
print(ols_b - lad_b)
print(lim_b - lad_b)

const       0.234264
lsales      0.015384
lmktval    -0.044394
ceoten      0.001995
ceotensq   -0.000224
dtype: float64
const       0.003831
lsales      0.004559
lmktval     0.000570
ceoten     -0.007122
ceotensq    0.000174
dtype: float64


C13.i The results are reported above

C13.ii 9 observations have studentized residuals above 1.96. Taking 5% of 177 and using the closest integer gives 9 and so this is consistent with expectations.

C13.iii lmktvalue sems to have gained at the expense of lsales. Tenure's effects are slightly diminished.

C13.iv The coefficient for lsales is closer to the limited regression than the full, but the coefficient for ceo tenure is closer to the full regression.

C13.v The statement is not demonstrated with this example. While lsalary was closer to the LAD estimate, the effects of ceo tenure (and the quadratic term) are are closer to OLS.